In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

from config import gkey

In [98]:
cities_df = pd.read_csv('resources/city_results.csv')
all_cities_df = pd.read_csv('resources/city_results.csv')

In [62]:
cities_df = cities_df.loc[(cities_df['Temperature (F)'] >= 70) \
                          & (cities_df['Temperature (F)'] <= 80) \
                          & (cities_df['Wind Speed (mph)'] <= 80) \
                          & (cities_df['Cloudiness (%)'] == 0)]

In [ ]:
cities_df['Hotel Lat'] = ""
cities_df['Hotel Lng'] = ""
cities_df['Hotel Name'] = ""
cities_df = cities_df.reset_index(drop=True)

In [72]:
def get_hotel_from_latlng(lat, lng):
    params = {
        "location": f"{lat},{lng}",
        "rankby": "distance",
        "keyword": "hotel",
        "types": "lodging",
        "key": gkey
    }
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    return requests.get(url, params=params)

In [80]:
for index, row in cities_df.iterrows():
    res = get_hotel_from_latlng(row['Latitude'],row['Longitude']).json()
    try:
        hotel = res["results"][0]
        cities_df.loc[index, 'Hotel Name'] = hotel["name"]
        cities_df.loc[index, 'Hotel Lat'] = hotel["geometry"]["location"]["lat"]
        cities_df.loc[index, 'Hotel Lng'] = hotel["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        cities_df.loc[index, 'Hotel Name'] = np.nan
        cities_df.loc[index, 'Hotel Lat'] = np.nan
        cities_df.loc[index, 'Hotel Lng'] = np.nan

In [85]:
cities_df = cities_df.dropna().reset_index(drop=True)
cities_df.head(10)

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name,Hotel Lat,Hotel Lng
0,Biloela,AU,-24.42,150.50,76.73,59,0,2.21,Biloela Caravan Park,-24.4084,150.496
1,Cabedelo,BR,-6.98,-34.83,78.80,83,0,6.93,Lua Mansa Pousada,-6.98145,-34.829
2,Arlit,NE,18.74,7.39,78.13,17,0,10.16,Hôtel Telwa Bungalow,18.739,7.38945
3,Pahrump,US,36.21,-115.98,72.63,18,0,15.01,Preferred RV Resort,36.2111,-115.982
4,Sambava,MG,-14.27,50.17,70.23,83,0,9.24,Hôtel Las Palmas,-14.2619,50.1653
5,Rambha,IN,19.52,85.10,75.63,81,0,3.96,Anwesha traders,19.5147,85.094
6,Erzin,TR,36.96,36.20,71.01,49,0,7.00,Artemis Otel,36.9527,36.2379
7,Saint-Joseph,RE,-21.37,55.62,71.60,68,0,10.29,La maison des hôtes à la Réunion- saint-joseph,-21.378,55.6182
8,Takhatgarh,IN,25.33,73.00,70.02,51,0,1.34,King Muthaliya Resort,25.3117,72.9862
9,Chandrakona,IN,22.73,87.52,72.59,62,0,3.85,SPOT ON 63175 Gitanjali Lodge & Hotel,22.7309,87.3391


In [158]:
gmaps.configure(api_key=gkey)

In [159]:
def make_info_box(name, city, country):
    name_dt = f"<dt>Name</dt><dd>{name}</dd>"
    city_dt = f"<dt>City</dt><dd>{city}</dd>"
    country_dt = f"<dt>Country</dt><dd>{country}</dd>"
    return f"<dl>{name_dt}{city_dt}{country_dt}</dl>"

In [152]:
info_boxes = []
for index, row in cities_df.iterrows():
    info_boxes.append(make_info_box(row['Hotel Name'], row['City'], row['Country']))

In [160]:
mark_layer = gmaps.marker_layer(cities_df[['Hotel Lat', 'Hotel Lng']], info_box_content=info_boxes)

In [177]:
hum_layer = gmaps.heatmap_layer(all_cities_df[['Latitude', 'Longitude']], weights=all_cities_df['Humidity (%)'])
hum_layer.max_intensity = 100
hum_layer.point_radius = 10

In [178]:
vmap = gmaps.figure(center=(0,0), zoom_level=2)
vmap.add_layer(mark_layer)
vmap.add_layer(hum_layer)
vmap

Figure(layout=FigureLayout(height='420px'))